Set-up

In [ ]:
import os
import sys
import json
import openai
from langchain.tools import tool
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.schema.agent import AgentFinish
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
sys.path.append('../..')

In [ ]:
# Setting your personal OPENAI Api Key
openai.api_key = os.environ['OPENAI_API_KEY']

Initializing Helpers

In [ ]:
ui_function_identifier = "This is a UI function"

In [ ]:
class design_logo_input(BaseModel):
    topic: str = Field(..., description="The topic for which the logo should be designed for. For example, if the topic were 'ice-cream shop' then the logo returned would be a logo fit for the ice-cream shop to put on their sign and merchandise.")

@tool(args_schema=design_logo_input)
def design_logo_concepts(topic: str) -> str:
    """Returns the url for a logo design that is based on the provided topic"""
    # Currently a very empty function
    return f"s3://designs/logo_{len(topic)%5+1}.jpg"

In [ ]:
class product_UI_input(BaseModel):
    path: str = Field(... , description=f"The path of the image for the UI page. If no image is wanted or required provide an empty string."

@tool(args_schema=product_UI_input)
def produce_image_UI(path: str) -> str:
    f"""{ui_function_identifier}. Produces the JSON for a general UI page about an image."""
    data = {"component":"Image", "props":{"url":path}}
    return  json.dumps(data)            

In [ ]:
tools = [design_logo_concepts, produce_image_UI]
functions = [format_tool_to_openai_function(f) for f in tools]

In [ ]:
cognitive_ui_instructions = "Your task is to take the information from the scratchpad and produce an output in the appropriate" +\
        " format. You have two choices, you can either produce a UI page by calling one of the UI functions provided to you (and" +\
        " returning the function call) or just output plain text like you are used to. The UI functions are functions that have" +\
        f" the phrase '{ui_function_identifier}' in their description, do not use functions that don't have this. If the appropriate" +\
        " UI function isn't available, use the closest fitting UI function, lack of appropriate function should not dissuade you" +\
        " from choosing the UI output option"

In [ ]:
tool_dictionary = {
        "design_logo_concepts": design_logo_concepts,
        "produce_image_UI": produce_image_UI
    }

Initializing Chain + Agent

In [ ]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant, don't be afraid to use the functions provided to you."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
def run_agent(user_input):
    intermediate_steps = []
    result = chain.invoke({
        "input": user_input,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps)
    })
    # Are we calling the functions? (If not we return)
    if isinstance(result, AgentFinish):
        return result
    tool = tool_dictionary[result.tool]
    observation = tool.run(result.tool_input)
    intermediate_steps.append((result, observation))
    # Cognitive UI Steps
    result = chain.invoke({
        "input": cognitive_ui_instructions,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps)
    })
    tool = tool_dictionary[result.tool]
    observation = tool.run(result.tool_input)
    return observation

Testing